In [1]:
import custom_coco_helper as helper
import os

In [2]:
# Constants
DATA_DIR = 'data'
DOWNLOAD_DIR = 'raw'
IMAGE_PATH = os.path.join(os.getcwd(), DATA_DIR, DOWNLOAD_DIR)

In [3]:
predictor = helper.MyPredictor(IMAGE_PATH)

In [4]:
categories = [
    {
        'id': 1,
        'name': 'people',
        'supercategory': 'people',
    },
]

In [5]:
next_seg_id = 1
annotations = []
imgs_info = []
my_instances = []
for i in range(2):
    img = predictor.read_im(i)
    results = predictor.inference(img)
    next_seg_id, instances = helper.MyInstance.create_instances(i, next_seg_id, predictor.get_img_path[i], results, keep_cats=[0])
    imgs_info.append(instances[0].coco_img_info())
    for instance in instances:
        my_instances.append(instance)
        annotation = instance.coco_annotation()
        if annotation is not None:
            annotations.append(annotation)